In [ ]:
print("ok")

In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#Extract text from pdf


def load_pdf_file(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    document = loader.load()
    return document



In [ ]:
extracted_data = load_pdf_file(data = 'data/')

In [ ]:
extracted_data

In [ ]:
# split the data into text chunks 

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
def downlaod_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embedding = downlaod_hugging_face_embeddings()

In [ ]:
query_result = embedding.embed_query("hello world")
print(len(query_result))

In [ ]:
pip install -U sentence-transformers

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_LQzrX_Lk3V5nRLHVYgmxJHrmsBqazdhPMYmzhsYRdf1p1eTcnsmC4LU8BVVgRzyaPreNr")

index_name = "synthmind"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(documents=text_chunks, embedding=embedding, index_name=index_name)

In [ ]:
# Existing index 

docsearch = PineconeVectorStore.from_existing_index(index_name=index_name,embedding=embedding)

In [ ]:
docsearch

In [ ]:
retriver = docsearch.as_retriever(search_type="similarity",search_kwarg={"k":3})

In [ ]:
retriver_docs= retriver.invoke("i am feeling bad")

In [ ]:
retriver_docs

In [ ]:
pip install -U langchain-ollama

In [ ]:
llm = ChatOllama(model="deepseek-coder-v2")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
"You are an assistant for question-answering tasks."
"Use the following pieces of retrieved context fto answer "
"the •question. If you don't know the answer, say that you"
"don't know. Use three sentences maximum and keep the"
"answer concise. "
"\n\n"
"{context}" 
)

prompt = ChatPromptTemplate.from_messages(
    [("system",system_prompt),
     ("human","{input}"),
    ]
)

In [ ]:
question_answering_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,question_answering_chain)

In [ ]:
from langchain_community.chat_models import ChatOllama

In [ ]:
response = rag_chain.invoke({"input":"feeling down"})
print(response["answer"])
